### Importing The Dataset

In [1]:
# importing packages
import pandas as pd
import numpy as np
from datetime import datetime

# Loading the csv file
bank_marketing = pd.read_csv("./Data/bank_marketing.csv")
bank_marketing.head(1)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Splitting The Dataset

In [506]:
# creating the client dataframe
client_column=['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']
client= bank_marketing[client_column].copy()
client.head(1)

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no


In [507]:
# creating the campaign dataframe
campaign_columns= ['campaign', 'duration', 'previous', 'y', 'poutcome', 'pdays']
campaign = bank_marketing[campaign_columns].copy()
campaign.head(1)

,campaign,duration,previous,y,poutcome,pdays
0,1,261,0,no,nonexistent,999


In [508]:
# creating the economics dataframe
economics_columns= ['emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']
economics = bank_marketing[economics_columns].copy()
economics.head(1)

,emp_var_rate,cons_price_idx,euribor3m,nr_employed
0,1.1,93.994,4.857,5191.0


### Data Cleaning

In [509]:
# Renaming some columns in the three dataframes
client.rename(columns= {"client_id" : "id"}, inplace= True)
client.columns

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')

In [510]:
campaign.rename(columns= {"duration" : "contact_duration", "previous":"previous_campaign_contacts", "y":"campaign_outcome", "poutcome":"previous_outcome", "campaign":"number_contacts" }, inplace= True)
campaign.columns

Index(['number_contacts', 'contact_duration', 'previous_campaign_contacts',
       'campaign_outcome', 'previous_outcome', 'pdays'],
      dtype='object')

In [511]:
economics.rename(columns={"euribor3m":"euribor_three_months", "nr_employed":"number_employed"}, inplace= True)
campaign.columns

Index(['number_contacts', 'contact_duration', 'previous_campaign_contacts',
       'campaign_outcome', 'previous_outcome', 'pdays'],
      dtype='object')

In [512]:
# changing "." in education column to "_"
client['education']= client['education'].str.replace(".", "_")
client.head(1)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no


In [513]:
# changing "unknown" to NumPy's null values
client['education']= client['education'].replace("unknown", np.nan)
client.education.unique()

array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [514]:
# removing "." in job column
client['job']= client['job'].str.replace(".", "")
client.head(1)
client.job.unique()

array(['housemaid', 'services', 'admin', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [515]:
# Converting "success" and "failure" in the "previous_outcome" column to binary (1 or 0)
campaign["previous_outcome"]= campaign["previous_outcome"].map({"success":1, "failure":0})

campaign.previous_outcome.unique()

array([nan,  0.,  1.])

In [516]:
# checking for unique values in campaign_outcome column
campaign.campaign_outcome.unique()

array(['no', 'yes'], dtype=object)

In [517]:
# Converting "yes" and "no" in the  "campaign_outcome" column to binary (1 or 0)
campaign["campaign_outcome"]= campaign["campaign_outcome"].map({"yes":1, "no":0})
campaign.campaign_outcome.unique()

array([0, 1])

In [518]:
# creating new column in campaign dataframe
#campaign["campaign_id"]=pd.series(1, index=)
campaign.insert(0, "campaign_id", [1]*41188, allow_duplicates=True)

In [519]:
campaign["campaign_outcome"]

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: campaign_outcome, Length: 41188, dtype: int64

In [520]:
# Create a datetime column called last_contact_date in campaign data frame

df= pd.DataFrame()
df=bank_marketing[['month','day']].copy()
df.insert(0, "year", [2022]*41188, allow_duplicates=True)

df["contact_date"]=df["year"].astype(str)+'-'+df["month"].str.capitalize()+'-'+df["day"].astype(str)
df["contact_date"]= df["contact_date"].astype('datetime64[ns]')
df["contact_date"]=pd.to_datetime(df["contact_date"]).dt.date
campaign["last_contact_date"]=df[["contact_date"]].copy()

In [521]:
campaign.head(1)

,campaign_id,number_contacts,contact_duration,previous_campaign_contacts,campaign_outcome,previous_outcome,pdays,last_contact_date
0,1,1,261,0,0,NaN,999,2022-05-13


In [522]:
campaign.insert(1, "client_id", client.id)

In [523]:
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [524]:
economics.head()

,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,1.1,93.994,4.857,5191.0
1,1.1,93.994,4.857,5191.0
2,1.1,93.994,4.857,5191.0
3,1.1,93.994,4.857,5191.0
4,1.1,93.994,4.857,5191.0


In [525]:
economics.insert(0, "client_id", client.id)

In [526]:
economics.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [527]:
# saving economics dataframe to csv
economics.to_csv("economics.csv", index=False)

In [528]:
# saving client dataframe to csv
client.to_csv("client.csv", index=False)

In [529]:
# saving campaign dataframe to csv
client.to_csv("campaign.csv", index=False)

### Designing The Database

In [530]:
# creating the client_table
client_table= """
CREATE TABLE client (
	id	SERIAL	PRIMARY KEY,
	age INTEGER,
	job TEXT,
	marital	TEXT,
	education TEXT,
	credit_default BOOLEAN,
	housing BOOLEAN,
	loan BOOLEAN	
);

\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER
"""

In [531]:
# creating the campaign_table
campaign_table= """
CREATE TABLE campaign (
	campaign_id	SERIAL	PRIMARY KEY,
    client_id SERIAL references client (id),
	number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
	previous_outcome BOOLEAN,
	campaign_outcome BOOLEAN,
	last_contact_date DATE	
);

\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER
"""



In [532]:
# creating the economics_table
economics_table= """
CREATE TABLE economics (
    client_id SERIAL references client (id),
	emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);

\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER
"""